In [ ]:
# **************************************************************************
# INF7370-Hiver 2024
# Travail pratique 1
# ===========================================================================
# ===========================================================================
# Indiquer votre nom ici
# ===========================================================================
# ===========================================================================

# ===========================================================================
# Le but de ce travail est de classifier les restaurants en 2 états (Fermeture définitive / Ouvert)
#
# Ce fichier consiste la troisième étape du travail -> entrainement des modèles de classification
# Dans ce fichier code, vous devez entrainer 5 modèles de classification sur les données préparées dans l'étape précédente.
# ===========================================================================

# ==========================================
# ======CHARGEMENT DES LIBRAIRIES===========
# ==========================================

# la librairie principale pour la gestion des données
import pandas as pd

# la librairie pour normalizer les données par Z-Score
from sklearn.preprocessing import StandardScaler

# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
# - Inclure ici toutes les autres librairies dont vous aurez besoin
# - Écrivez en commentaire le rôle de chaque librairie
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

# les librairies pour la visualisation (courbes ROC et AUC)
import scikitplot as skplt
import matplotlib.pyplot as plt

# la librairie pour diviser les données en ensembles d'entraînement et de test
from sklearn.model_selection import train_test_split

# les librairies pour les modèles de classification
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier

# les librairies pour calculer diverses métriques d'évaluation des modèles
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score

# la librairie pour la sélection des caractéristiques basée sur le critère du gain d'information mutuel
from sklearn.feature_selection import SelectKBest, mutual_info_classif

# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>


# ==========================================
# ===============VARIABLES==================
# ==========================================

# l'emplacement des données sur le disque
# Note: Il faut placer le dossier "donnees"  contenant les 8 fichiers .csv dans le même endroit que les fichiers de code
data_path = "donnees/"

# ==========================================
# ===============FONCTIONS==================
# ==========================================

    # """
    # La fonction entraîne plusieurs modèles sur un jeu de données donné et retourne une liste des modèles entraînés.

    # :param models: Une liste de tuples, où chaque tuple contient le nom du modèle et l'objet modèle lui-même.
    # Par exemple, [("Modèle 1", modele1), ("Modèle 2", modele2)]
    # :param x_train: Les données d'entraînement, qui consistent en les caractéristiques d'entrée utilisées pour entraîner les modèles.
    # C'est un objet de type matrice ou similaire avec la forme (n_samples, n_features), où n_samples est le nombre
    # d'échantillons dans les données d'entraînement et n_features est le nombre de caractéristiques d'entrée pour chaque échantillon.
    # :param y_train: Le paramètre y_train est la variable cible ou la variable dépendante. Elle représente la variable
    # que nous essayons de prédire ou de classifier.
    # :return: Une liste de tuples, où chaque tuple contient le nom d'un modèle et le modèle entraîné lui-même.
    # """
def train_models(models, x_train, y_train):
    """
    Entraîne une liste de modèles de classification donnée.
    :param models: Liste de tuples (nom_du_modèle, instance_du_modèle)
    :param x_train: Données d'entraînement
    :param y_train: Étiquettes d'entraînement
    :return: Liste de modèles entraînés
    """
    trained_models = []
    for name, model in models:
        model.fit(x_train, y_train)
        trained_models.append((name, model))
    return trained_models

    # """
    # La fonction "calculate_metrics" calcule diverses métriques telles que le taux de vrais positifs (TP Rate),
    # le taux de faux positifs (FP Rate), la F-mesure et l'AUC pour évaluer la performance d'un modèle de classification binaire.

    # :param y_test: Les étiquettes réelles de l'ensemble de test.
    # :param y_pred: Les étiquettes prédites pour l'ensemble de test.
    # :return: un dictionnaire contenant les métriques suivantes : TP Rate, FP Rate, F-mesure et AUC.
    # """
def calculate_metrics(y_test, y_pred, y_probas):
    """
    Calcule et retourne les métriques d'évaluation pour les prédictions données, en utilisant les probabilités pour l'AUC.
    :param y_test: Étiquettes réelles
    :param y_pred: Prédictions du modèle
    :param y_probas: Probabilités de la classe positive
    :return: Dictionnaire de métriques
    """
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    metrics = {
        "TP Rate": tp / (tp + fn),
        "FP Rate": fp / (fp + tn),
        "F-mesure": f1_score(y_test, y_pred),
        "AUC": roc_auc_score(y_test, y_probas[:, 1])  # Utiliser les probabilités de la classe positive
    }
    return metrics

    # """
    # La fonction `evaluate_models` prend une liste de modèles entraînés, les données de test et les étiquettes de test, et évalue chaque modèle en calculant des métriques, en traçant des courbes ROC et en affichant des matrices de confusion.

    # :param trained_models: Le paramètre "trained_models" est un dictionnaire ou une liste de modèles d'apprentissage automatique entraînés. Chaque modèle est associé à un nom ou identifiant unique.
    # :param x_test: Le paramètre `x_test` représente les caractéristiques d'entrée du jeu de données de test. C'est un objet matriciel ou similaire à un tableau avec la forme (n_samples, n_features), où n_samples est le nombre d'échantillons dans le jeu de données de test et n_features est le nombre de caractéristiques pour chaque échantillon.
    # :param y_test: Le paramètre `y_test` est les étiquettes réelles ou les valeurs cibles pour le jeu de données de test. Il est utilisé pour évaluer la performance des modèles entraînés en comparant les étiquettes prédites avec les étiquettes réelles.
    # """
def evaluate_models(trained_models, x_test, y_test):
    """
    Évalue les modèles entraînés et affiche les graphiques ROC et la matrice de confusion.
    """
    for name, model in trained_models:
        y_probas = model.predict_proba(x_test)
        y_pred = model.predict(x_test)

        metrics = calculate_metrics(y_test, y_pred, y_probas)
        print(f'Modèle: {name}')
        for metric_name, metric_value in metrics.items():
            print(f"{metric_name}: {metric_value}")

        skplt.metrics.plot_roc(y_test, y_probas, plot_micro=False, plot_macro=False)
        plt.title(f'Courbe ROC - {name}')
        plt.show()

        skplt.metrics.plot_confusion_matrix(y_test, y_pred, normalize=True)
        plt.title(f'Matrice de confusion - {name}')
        plt.show()

        print('******************************\n')

# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
# - Inclure ici toutes les autres variables globales dont vous aurez besoin
# - Écrivez en commentaire le rôle de chaque variable
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

# Liste des modèles à entraîner
models = [
    ("Arbre de décision", DecisionTreeClassifier()),
    ("Forêt aléatoire", RandomForestClassifier()),
    ("Naive Bayes", GaussianNB()),
    ("Bagging", BaggingClassifier()),
    ("AdaBoost", AdaBoostClassifier())
]

# ==========================================
# ====CHARGEMENT DES DONNÉES EN MÉMOIRE=====
# ==========================================

# Charger en mémoire les features préparées dans la deuxième étape (pré-traités)
features = pd.read_csv(data_path + "features_finaux.csv")

# ==========================================
# INITIALIZATION DES DONNÉES ET DES ÉTIQUETTES
# ==========================================

# Initialisation des données et des étiquettes
x = features.copy() # "x" contient l'ensemble des données d'entrainement
y = x["ferme"]      # "y" contient les étiquettes des enregistrements dans "x"

# Elimination de la colonne classe (ferme) des features
x = x.drop('ferme', axis=1)

# Sauvegardez les noms des colonnes de 'x' pour une utilisation ultérieure
feature_names = x.columns

# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
#                      QUESTION 1
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
#  - Normaliser les données en utilisant Z-score (StandardScaler dans Scikit-learn)
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

sc = StandardScaler()
x = sc.fit_transform(x)

# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
#                      QUESTION 2
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
# - Divisez les données en deux lots (entrainement et test)
# (indiquer dans votre rapport le pourcentage des données de test que vous avez utilisé)
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
#                      QUESTION 3
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
# - Entrainez 5 modèles de classification sur l'ensemble de données normalisées (avec tous les features)
#   1 - Arbre de decision
#   2 - Forêt d’arbres décisionnels (Random Forest)
#   3 - Classification bayésienne naïve
#   4 - Bagging
#   5 - AdaBoost
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

# Entraînement des modèles
trained_models = train_models(models, x_train, y_train)

# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
#                      QUESTION 4
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
# - Afficher les resultats sur les données test de chaque algorithm entrainé avec tous les features
#   1- Le taux des vrais positifs (TP Rate) – de la classe Restaurants fermés définitivement.
#   2- Le taux des faux positifs (FP Rate) – de la classe Restaurants fermés définitivement.
#   3- F-measure de la classe Restaurants fermés définitivement.
#   4- La surface sous la courbe ROC (AUC).
#   5- La matrice de confusion.
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

# Évaluation des modèles
evaluate_models(trained_models, x_test, y_test)

# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
#                      QUESTION 5
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
# - Selectionnez les tops 10 features
#
# Vous devez identifier les 10 meilleurs features en utilisant la mesure du Gain d’information (Mutual Info dans scikit-learn).
# Afficher les 10 meilleurs features dans un tableau (par ordre croissant selon le score obtenu par le Gain d'information).
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

# Sélection des 10 meilleurs features avec Mutual Information
selector = SelectKBest(mutual_info_classif, k=10)
x_selected = selector.fit_transform(x, y)

# Récupération des scores pour chaque feature
feature_scores = selector.scores_

# Création d'un DataFrame pour mieux visualiser les scores et les noms des features
feature_scores_df = pd.DataFrame({'Feature': feature_names, 'Score': feature_scores})

# Trier les features selon leur score de Gain d'information
# Return the first n rows ordered by columns in descending order.
# This method is equivalent to df.sort_values(columns, ascending=False).head(n), but more performant.
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.nlargest.html
top_10_features = feature_scores_df.nlargest(10, 'Score')

# Affichage des 10 meilleurs features
print("Les 10 meilleurs caractéristiques sélectionnés selon le Gain d'information sont :")
print(top_10_features)
print("\n", "-"*80, "\n")

# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
#                      QUESTION 6
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
# - Entrainez 5 modèles de classification sur l'ensemble de données normalisées avec seulement les top 10 features selectionnés.
#   1 - Arbre de decision
#   2 - Forêt d’arbres décisionnels (Random Forest)
#   3 - Classification bayésienne naïve
#   4 - Bagging
#   5 - AdaBoost
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

# Sélection des indices des 10 meilleurs caractéristiques pour la réduction des données
top_10_features_indices = selector.get_support(indices=True)

# Création de nouveaux ensembles d'entraînement et de test avec seulement les 10 meilleurs caractéristiques
x_train_reduced = x_train[:, top_10_features_indices]
x_test_reduced = x_test[:, top_10_features_indices]

# Réutilisation des fonctions pour entraîner et évaluer les modèles réduits
trained_models_reduced = train_models(models, x_train_reduced, y_train)

# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
#                      QUESTION 7
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
# - Afficher les resultats sur les données test de chaque algorithm entrainé avec les top 10 features
#   1- Le taux des vrais positifs (TP Rate) – de la classe Restaurants fermés définitivement.
#   2- Le taux des faux positifs (FP Rate) – de la classe Restaurants fermés définitivement.
#   3- F-measure de la classe Restaurants fermés définitivement.
#   4- La surface sous la courbe ROC (AUC).
#   5- La matrice de confusion.
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

# Évaluation des modèles réduits
evaluate_models(trained_models_reduced, x_test_reduced, y_test)